# 1. Flats above average


## 1.1 Flats above median
What's the probability a flat has a price given it's features?

There is still to few data to compute $\int\, dprice\_sqm price\_smq\, P(price\_sqm|Arr,Meuble,sqm)$, so I use the simpler conditional probabilities $P(Arr,Meuble)$ or $P(Arr,sqm)$ that need less classes.

A similar analysis can be tried with a ``naive Bayes`` or a ``logit`` model.. TODO


In [2]:
import qgrid
import pandas as pd
df = pd.read_pickle('data/lbc_pandas.pkl')


In [5]:
df['Arr_Meu_cls']=0
df['Arr_Sqm_cls']=0
def average_flat(to):
    def f(group):
        mean_price = group.price_sqm.median()
        group[to] [group.price_sqm>mean_price]=1
    #     group.Arr_meu_cls = group.price_sqm>mean_price.astype(int)
        return group
    return f


df = df.groupby(('Arrondissement','Meuble')).apply(average_flat('Arr_Meu_cls'))
df = df.groupby(('Arrondissement','sqm_bin')).apply(average_flat('Arr_Sqm_cls'))
careof = ['LoyerMensuel','Surface','price_sqm','Arrondissement','Meuble','ClasseEnergie_lit','url']
qgrid.show_grid(df[df.Arr_Meu_cls>0][careof])
# print df[df.Arr_Meu_cls>0].shape[0]*1./df.shape[0]
    

In [4]:
from IPython.display import HTML
import urllib2

style=open('style.css','r').read()
HTML("""
<style>{0}</style>
""".format(style))